# Compaction Solver

This notebook solves the viscous compaction problem in one spatial dimension (vertical). 

For a brief overview of the problem and numerical method, see the notebook notes.ipynb.

The code requires [FEniCSx](https://fenicsproject.org). The notebook can be run 
through a [Docker](https://www.docker.com) container with the command:

`docker run --init -ti -p 8888:8888 -v $(pwd):/home/fenics/shared -w /home/fenics/shared dolfinx/lab:stable`

Various imports:

In [ ]:
%load_ext autoreload
%autoreload 2
# add path to code
import sys
sys.path.insert(0, '../source')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import numpy as np
from dolfinx.fem import Constant, Function, FunctionSpace, Expression
from dolfinx.mesh import create_interval
from misc import interp, sat, C, temp, K
from mpi4py import MPI
from params import nt, nz, phi0, delta, L, Tf, Tm
from petsc4py import PETSc
from solvers import vel_solve, full_solve
from ufl import FiniteElement, MixedElement, Dx

In [ ]:
print('compaction length delta = '+'{:.2e}'.format(delta)+' m')
# print('density = '+'{:.2e}'.format(rho_b)+' kg/m^3')

Define domain:

In [ ]:
# generate mesh, initial domain is scaled height of 1 
domain = create_interval(MPI.COMM_WORLD,nz,[-L,0])

Define initial porosity:

In [ ]:
P1 = FiniteElement('P',domain.ufl_cell(),1)     
element = P1*P1
V = FunctionSpace(domain,element)   
initial = Function(V)
initial.sub(1).interpolate(lambda x:phi0+0*x[0])

Solve the momentum balance for the initial porosity to obtain the initial velocity:

In [ ]:
Gamma = 0 # mass supply rate to ice lens

In [ ]:
w = vel_solve(domain,phi0,Gamma)
initial.sub(0).interpolate(w)

In [ ]:
V0 = FunctionSpace(domain, ("CG", 1)) 
wz_i = Function(V0)
wz_i.interpolate(Expression(Dx(w,0), V0.element.interpolation_points()))

z = domain.geometry.x[:,0]
w_ = w.x.array
wz = wz_i.x.array
T = temp(z)
S_ = sat(T)
K_ = K(S_)

plt.figure(figsize=(10,6))
plt.subplot(141)
plt.plot(T,z)
plt.axvline(x=0,color='k',linestyle='--',linewidth=1)
plt.xlabel(r'$\theta$',fontsize=16)
plt.ylabel(r'$z$',fontsize=16)
plt.subplot(142)
plt.plot(w_,z)
plt.xlabel(r'$w_\mathrm{s}$',fontsize=16)
plt.yticks([])
plt.subplot(143)
plt.plot(wz,z,'--')
plt.axvline(x=0,color='k',linestyle='--',linewidth=1)
plt.xlabel(r'$\frac{\partial w_\mathrm{s}}{\partial z}$',fontsize=16)
plt.yticks([])
plt.subplot(144)
plt.plot(S_,z)
plt.xlim(-0.1,1.1)
plt.xlabel(r'$S$',fontsize=16)
plt.gca().yaxis.set_label_position("right")
plt.gca().yaxis.tick_right()
plt.ylabel(r'$z$',fontsize=16)
plt.show()
plt.close()

In [ ]:
V = FunctionSpace(domain, ("CG", 1))
wi = Function(V)    # initial ice velocity

In [ ]:
m = 0#-1e-1 # melting rate

In [ ]:
ws,phi, wi, z = full_solve(domain,initial,m,Gamma)

In [ ]:
ind = np.arange(0,nt,int(nt/10))
colors = pl.cm.plasma_r(np.linspace(0,1,ind.size))
plt.figure(figsize=(10,6))
plt.subplot(131)
for i in range(ind.size):
    plt.plot(ws[ind[i],:],z[ind[i],:],color=colors[i])
plt.xlabel(r'$w_\mathrm{s}$',fontsize=16)
plt.ylabel(r'$z$',fontsize=16)
plt.subplot(132)
for i in range(ind.size):
    plt.plot(phi[ind[i],:],z[ind[i],:],color=colors[i])
plt.plot(S_,z[0,:],'k--')    
plt.xlim(-0.1,1)
plt.xlabel(r'$\phi$, $S$',fontsize=16)
plt.yticks([])
plt.subplot(133)
for i in range(ind.size):
    plt.plot(0*wi[ind[i],:],z[ind[i],:],color='w')
    plt.plot(wi[ind[i],:][S_>0],z[ind[i],:][S_>0],color=colors[i])  
plt.xlabel(r'$w_\mathrm{i}$',fontsize=16)
plt.gca().yaxis.set_label_position("right")
plt.gca().yaxis.tick_right()
plt.ylabel(r'$z$',fontsize=16)
plt.show()
plt.close()